In [1]:
import pandas as pd
import numpy as np
import random
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle

In [2]:
# --- Génération des données synthétiques ---
num_samples = 2000
data = []

for _ in range(num_samples):
    duree_anticipation_minutes = random.randint(5, 120)
    est_fin_semaine = random.choice([0, 1]) # 1 pour Vendredi, 0 sinon
    nb_hs_recentes_heures = random.randint(0, 15)
    charge_travail_jour = round(random.uniform(0.1, 1.0), 2)
    
    # Règle métier pour la décision réelle (vérité terrain)
    decision_reelle = 0 # 0 = Justification requise (par défaut)
    
    # On tolère si l'employé est parti un peu avant un vendredi ET qu'il a fait des HS récemment
    if duree_anticipation_minutes <= 30 and est_fin_semaine == 1 and nb_hs_recentes_heures > 5:
        decision_reelle = 1 # 1 = Tolérer
    # On tolère aussi les toutes petites sorties anticipées si la charge de travail est faible
    elif duree_anticipation_minutes <= 10 and charge_travail_jour < 0.4:
        decision_reelle = 1

    data.append([
        duree_anticipation_minutes,
        est_fin_semaine,
        nb_hs_recentes_heures,
        charge_travail_jour,
        decision_reelle
    ])

# Création du DataFrame
columns = [
    'duree_anticipation_minutes', 
    'est_fin_semaine', 
    'nb_hs_recentes_heures', 
    'charge_travail_jour', 
    'decision_reelle'
]
df_sortie = pd.DataFrame(data, columns=columns)

# Sauvegarde du dataset
df_sortie.to_csv('dataset_sortie_anticipee.csv', index=False)

print("Dataset généré et sauvegardé.")
df_sortie.head()

Dataset généré et sauvegardé.


,duree_anticipation_minutes,est_fin_semaine,nb_hs_recentes_heures,charge_travail_jour,decision_reelle
0,64,1,10,0.60,0
1,45,0,8,0.43,0
2,14,1,9,0.87,1
3,48,0,13,0.61,0
4,105,0,11,0.30,0


In [3]:
# --- Entraînement du modèle ---
features = [
    'duree_anticipation_minutes', 
    'est_fin_semaine', 
    'nb_hs_recentes_heures', 
    'charge_travail_jour'
]
target = 'decision_reelle'

X = df_sortie[features]
y = df_sortie[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Création et entraînement du modèle LightGBM
model_sortie = lgb.LGBMClassifier(random_state=42)
model_sortie.fit(X_train, y_train)

print("Modèle de sortie anticipée entraîné !")

# Évaluation
y_pred = model_sortie.predict(X_test)
print("\nRapport de classification :")
print(classification_report(y_test, y_pred))

[LightGBM] [Info] Number of positive: 133, number of negative: 1467
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000681 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 227
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.083125 -> initscore=-2.400626
[LightGBM] [Info] Start training from score -2.400626
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [4]:
# --- Sauvegarde du modèle ---
model_filename = 'model_sortie_anticipee.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(model_sortie, file)

print(f"Modèle sauvegardé dans '{model_filename}'")

Modèle sauvegardé dans 'model_sortie_anticipee.pkl'
